In [1]:
import torch
import torchvision
import numpy as np
import torchvision.transforms as transforms
import sys
from d2lzh_pytorch import *

# 3.6.1 获取数据
**使用Fashion-MNIST数据集，设置批量大小为256**

In [2]:
batch_size = 256
train_iter, test_iter = load_data_fashion_mnist(batch_size)

C:\Users\28663\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# 3.6.2 初始化模型参数
**模型输入向量的长度为 28\*28=784，由于图像有 10个类别，单层神经网络输出层的输出个数为 10，因此 softmax回归的权重和偏差参数分别为 784\*10和 1\*10的矩阵**

In [3]:
num_inputs = 784
num_outputs = 10

W = torch.tensor(np.random.normal(0, 0.01, (num_inputs, num_outputs)), dtype=torch.float)
b = torch.zeros(num_outputs, dtype=torch.float)

#需要模型参数梯度
W.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

# 3.6.3 实现softmax运算
**首先描述一下如何对多维 tensor按维度操作，我们可以只对其中同一列（dim=0）或同一行（dim=1）的元素求和，并在结果中保留行和列这两个维度（keepdim=True）**

In [4]:
X = torch.tensor([[1,2,3], [4,5,6]])
print(X.sum(dim=0, keepdim=True))
print(X.sum(dim=1, keepdim=True))

tensor([[5, 7, 9]])
tensor([[ 6],
        [15]])


为了表达样本预测各个输出的概率，softmax运算会先通过 exp函数对每个元素做指数运算，再对 exp矩阵同行元素求和，最后令矩阵每行各元素与该元素之和相除，这样一来，最终得到的矩阵每行元素和为 1且非负。$$$$
**softmax运算的输出矩阵中的任意一行元素代表了一个样本在各个输出类别上的预测概率**

In [5]:
def softmax(X):
    X_exp = X.exp()
    partition = X_exp.sum(dim=1, keepdim=True)
    return X_exp / partition

X = torch.rand((2, 5))
X_prob = softmax(X)
print(X_prob, X_prob.sum(dim=1))

tensor([[0.1514, 0.2050, 0.3477, 0.1542, 0.1417],
        [0.1660, 0.2269, 0.1687, 0.1876, 0.2508]]) tensor([1.0000, 1.0000])


# 3.6.4 定义模型
**这里通过view函数将每张原始图像改成长度为num_inputs的向量**

In [6]:
def net(X):
    return softmax(torch.mm(X.view((-1, num_inputs)), W) + b)

# 3.6.5 定义损失函数
**softmax回归使用的交叉熵损失函数。使用变量y_hat是2个样本在3个类别的预测概率，变量y是这2个样本的标签类别。通过使用 gather 函数，得到两个样本的标签的预测概率**

In [7]:
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y = torch.LongTensor([0, 2])
y_hat.gather(1, y.view(-1, 1))

tensor([[0.1000],
        [0.5000]])

**gather函数提取数据时主要靠dim和index这两个参数：**
* dim=1时，将input看为nx1阶矩阵，index看成kx1阶级矩阵，取index每行元素对Input中每行进行列索引
* dim=0时，将input看为1xn阶矩阵，index看成1xk阶级矩阵，取index每列元素对Input中每列进行行索引

In [21]:
#实现交叉熵函数
def cross_entropy(y_hat, y):
    # 假设 y：256*10，对应每行元素非0即1，则只需要计算y为1的项即可，其他可以省略，得到交叉熵损失计算公式简化形式如下：
    # gather函数根据 y.view(-1, 1) 产生的index，只计算对正确类别的预测概率，以此计算损失
    return -torch.log(y_hat.gather(1, y.view(-1, 1)))

# 3.6.6 计算分类准确率
**给定一个类别的预测概率分布 y_hat，将概率预测最大的类别作为输出。**
* 相等条件判断式 (y_hat.argmax(dim=1)==y) 是一个类型为ByteTensor的Tensor，使用float()将其转换为值为0（相等为假）或1（相等为真）的浮点型tensor

In [9]:
def accuracy(y_hat, y):
    return (y_hat.argmax(dim=1)==y).float().mean().item()

print(accuracy(y_hat, y))

0.5


In [10]:
# 类似的评价一下模型net在数据集data_iter上的准确率
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        acc_sum += (net(X).argmax(dim=1)==y).float().sum().item()
        n += y.shape[0]
    return acc_sum / n

In [11]:
print(evaluate_accuracy(test_iter, net))

0.0971


# 3.6.7 训练模型
**使用小批量随机梯度下降来优化模型的损失函数**

In [20]:
num_epochs, lr = 5, 0.1

def train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, params=None, lr=None, optimizer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X, y in train_iter:
            y_hat = net(X)  # y_hat:256*10    y:(256,)
            l = loss(y_hat, y).sum() # 计算交叉熵损失
            
            
            #梯度清零
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
                    
            l.backward()
            if optimizer is None:
                sgd(params, lr, batch_size)
            else:
                optimizer.step()
                
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              %(epoch+1, train_l_sum / n, train_acc_sum / n, test_acc))
        
train_ch3(net, train_iter, test_iter, cross_entropy, num_epochs, batch_size, [W, b], lr)

torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])


KeyboardInterrupt: 